# TM10007 Assignment ADNI Dataset
Bram Gerritse 4462599  
Saskia Bijl 4439457  
Karan Ramsodit  
Enzo Kerkhof 44488555

### Install git repo:

In [67]:
# Run this to use from colab environment to install git repo
!pip install -q --upgrade git+https://github.com/Enzo-Kerkhof/TM10007_PROJECT.git@Enzo_testing

# Use the @ to determine what branch to install. Don't use to install master.

### Import used libraries

In [0]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import decomposition
from sklearn import neighbors
from sklearn import svm


### Data loading and splitting

In [0]:
# Import premade data loading function
from adni.load_data import load_data

data = load_data()

# Drop labels from dataframe
X = data.drop(['label'],axis=1)

# Binerize labels AD = 1, CN = 0
Y = preprocessing.label_binarize(data['label'], ['CN', 'AD'])
Y = [i[0] for i in Y]

# Split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.5)

### Missing data handeling

In [0]:
# Features with a lot of zero's?
# Samples with a lot of zero's?



### Feature scaling

Wat is hier de beste optie?

In [0]:
# Scale the data to be normal
scaler = preprocessing.RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Feature selection

Alleen PCA of eerst selecton én PCA?

In [0]:
# Perform a PCA
pca = decomposition.PCA(n_components=5)
pca.fit(X_train_scaled)
X_train_pca = pca.transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

### Fit classifier

Hoe ingewikkeld willen we de classifier? Wat past bij onze data?

In [72]:
# Fit kNN
knn = neighbors.KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_pca, y_train)
score_train_knn = knn.score(X_train_pca, y_train)
score_test_knn = knn.score(X_test_pca, y_test)

# Fit SVM
svmpoly = svm.SVC(kernel='poly', degree=3, gamma='scale')
svmpoly.fit(X_train_pca, y_train)
score_train_svm = svmpoly.score(X_train_pca, y_train)
score_test_svm = svmpoly.score(X_test_pca, y_test)

# Print result
print(f"Training result KNN: {score_train_knn}")
print(f"Test result KNN: {score_test_knn}")
print(f"Training result svm: {score_train_svm}")
print(f"Test result svm: {score_test_svm}")

Training result KNN: 0.7939110070257611
Test result KNN: 0.7593457943925234
Training result svm: 0.6088992974238876
Test result svm: 0.6051401869158879
